In [1]:
from ltr.clickmodels.session import build

# Sessions for search queries "rambo" and "star trek"
sessions = build([
      ('rambo',     ((1, True), (2, False), (3, True), (0, False))),
      ('star trek', ((5, False), (2, True), (3, True), (0, False))),
      ('rambo',     ((1, False), (2, False), (3, True), (0, False))),
      ('star trek', ((1, False), (2, False), (3, False), (9, True))),
      ('rambo',     ((9, False), (2, False), (1, True), (0, True))),
      ('star trek', ((6, True), (2, False), (3, True), (1, False))),
      ('rambo',     ((7, False), (4, True), (1, False), (3, False))),
      ('star trek', ((8, True), (2, False), (3, True), (1, False))),
      ('rambo',     ((1, False), (4, True), (2, False), (3, False))),
      ('star trek', ((7, True), (4, False), (5, True), (1, True)))
    ])

In [2]:
def by_query(judgment_dict):
    from collections import defaultdict
    by_query_ranked = defaultdict(list)
    
    for (query, doc), score in judgment_dict.items():
        by_query_ranked[query].append((doc,score))
        
    for query, items in by_query_ranked.items():
        items.sort(key=lambda score_doc: score_doc[1], reverse=True)
        
    return by_query_ranked

def just_doc_ids(docs):
    return [doc[0] for doc in docs]

# COEC 
(Clicks Over Expected Clicks)

Clicks relative to CTR at that position. > 1 means above average, < 1 means below average

In [3]:
from ltr.clickmodels.coec import coec

ctrs_by_rank = [0.15,0.15,0.15,0.03]

model = coec(ctrs_by_rank, sessions)
coec_judgments = by_query(model.coecs)

In [4]:
coec_judgments

defaultdict(list,
            {'rambo': [(0, 11.11111111111111),
              (4, 6.666666666666667),
              (3, 5.555555555555555),
              (1, 2.6666666666666665),
              (2, 0.0),
              (9, 0.0),
              (7, 0.0)],
             'star trek': [(9, 33.333333333333336),
              (6, 6.666666666666667),
              (8, 6.666666666666667),
              (7, 6.666666666666667),
              (3, 5.0),
              (1, 4.166666666666667),
              (5, 3.3333333333333335),
              (2, 1.6666666666666667),
              (0, 0.0),
              (4, 0.0)]})

# PBM

(Position-Based model)

We learn probability of a rank being examined and the attractiveness of a query-doc pair

In [5]:
from ltr.clickmodels.pbm import position_based_model
model = position_based_model(sessions, rounds=50)
pbm_judgments = by_query(model.attracts)

In [6]:
model.ranks

[0.9987182876053634,
 0.9999889839385798,
 0.9999999990640687,
 0.44523038207622834]

In [7]:
pbm_judgments['rambo']

[(4, 1.0),
 (3, 0.9989838560294817),
 (0, 0.747358098424573),
 (1, 0.4003830084292817),
 (9, 4.113007892053103e-81),
 (7, 4.113007892053103e-81),
 (2, 4.076400940523653e-131)]

### Compare

In [8]:
from ltr.helpers.tau import tau
from ltr.helpers.tau import avg_tau

rambo1 = just_doc_ids(coec_judgments['rambo'])
rambo1


[0, 4, 3, 1, 2, 9, 7]

In [9]:
rambo2 = just_doc_ids(pbm_judgments['rambo'])
rambo2


[4, 3, 0, 1, 9, 7, 2]

In [10]:
avg_tau(rambo1, rambo2, at=7)

-0.21632653061224488

In [11]:
star_trek1 = just_doc_ids(coec_judgments['star trek'])
star_trek1

[9, 6, 8, 7, 3, 1, 5, 2, 0, 4]

In [12]:
star_trek2 = just_doc_ids(pbm_judgments['star trek'])
star_trek2

[9, 6, 8, 7, 3, 5, 1, 2, 0, 4]

In [13]:
avg_tau(star_trek1, star_trek2, at=10)

0.9066666666666666

# UBM

(User Browse Model)

Variant of PBM model that computes examination as a function of this position and the last clicked position.

In [14]:
from ltr.clickmodels.ubm import user_browse_model

# Sessions for search queries "rambo" and "star trek"
sessions2 = build([
      ('rambo',     ((1, True), (2, False))),
      ('star trek', ((5, False), (2, True))),
      ('rambo',     ((1, True), (2, False))),
      ('star trek', ((5, False), (2, True)))
    
    ])

model = user_browse_model(sessions2, rounds=50)
model.attracts

defaultdict(<function ltr.clickmodels.ubm.Model.__init__.<locals>.<lambda>()>,
            {('rambo', 2): 0.0655509140827497,
             ('star trek', 5): 0.0,
             ('rambo', 1): 1.0,
             ('star trek', 2): 1.0})

In [15]:
model.ranks

defaultdict(<function ltr.clickmodels.ubm.Model.__init__.<locals>.<lambda>()>,
            {(0, 1): 0.0025465408240250265,
             (-1, 0): 0.9999999999999993,
             (-1, 1): 1.0})

# Cascade Model

A variant of the model that computes relevance based on clicks over times the article appeared at or before the first click

In [16]:
from ltr.clickmodels.cascade import cascade_model
model = cascade_model(sessions)

In [17]:
model.attracts

defaultdict(<function ltr.clickmodels.cascade.Model.__init__.<locals>.<lambda>()>,
            {('rambo', 1): 0.5,
             ('star trek', 5): 0.0,
             ('star trek', 2): 0.5,
             ('rambo', 2): 0.0,
             ('rambo', 3): 1.0,
             ('star trek', 1): 0.0,
             ('star trek', 3): 0.0,
             ('star trek', 9): 1.0,
             ('rambo', 9): 0.0,
             ('star trek', 6): 1.0,
             ('rambo', 7): 0.0,
             ('rambo', 4): 1.0,
             ('star trek', 8): 1.0,
             ('star trek', 7): 1.0})

# SDBN

Simplified version of the Dynamic Bayesian Network, that computes satisfaction and attractiveness. Satisfaction is the times this item was clicked relative to all sessions that item was clicked. Attractiveness how often this item was clicked relative to all sessions with that query/doc

In [18]:
from ltr.clickmodels.sdbn import sdbn
model = sdbn(sessions)

In [19]:
model.attracts

defaultdict(<function ltr.clickmodels.sdbn.Model.__init__.<locals>.<lambda>()>,
            {('rambo', 3): 1.0,
             ('rambo', 2): 0.0,
             ('rambo', 1): 0.5,
             ('star trek', 3): 0.75,
             ('star trek', 2): 0.25,
             ('star trek', 5): 0.5,
             ('star trek', 9): 1.0,
             ('star trek', 1): 0.5,
             ('rambo', 0): 1.0,
             ('rambo', 9): 0.0,
             ('star trek', 6): 1.0,
             ('rambo', 4): 1.0,
             ('rambo', 7): 0.0,
             ('star trek', 8): 1.0,
             ('star trek', 4): 0.0,
             ('star trek', 7): 1.0})

In [20]:
model.satisfacts

defaultdict(<function ltr.clickmodels.sdbn.Model.__init__.<locals>.<lambda>()>,
            {('rambo', 3): 1.0,
             ('rambo', 1): 0.0,
             ('star trek', 3): 1.0,
             ('star trek', 2): 0.0,
             ('star trek', 5): 0.0,
             ('star trek', 9): 1.0,
             ('star trek', 1): 1.0,
             ('rambo', 0): 1.0,
             ('star trek', 6): 0.0,
             ('rambo', 4): 1.0,
             ('star trek', 8): 0.0,
             ('star trek', 7): 0.0})